In [1]:
import pandas as pd 
import numpy as np 
import os 
import sys 
import re 
import json 
import ast

In [3]:
rounds = 10

In [4]:
sasi = pd.read_csv("../../data/conv-trs/multi-agent/results/sasi_results.csv")

In [5]:
sasi_gemini = sasi[['config_id', 'config', 'query', 'response_Gemini2Flash']]

In [6]:
# mami_dir = "../../data/conv-trs/multi-agent/results/prototype_updated_relevance/gemini2flash/states/"
# mami_dir = "../../data/conv-trs/multi-agent/results/prototype/gemini2flash/states/" # for default

mami_dir = "../../data/conv-trs/multi-agent/results/prompt_ranking_aggressive/gemini2point5flash/states/"

mami_results = [] 

for file in os.listdir(mami_dir):
    config_id = (file.split("states_")[1]).split(".json")[0]
    
    with open(f"{mami_dir}{file}", "r") as f: 
        data = json.load(f)

    final_offer = data[str(rounds-1)]["collective"]["offers"]
    final_city_scores = data[str(rounds-1)]["collective"]["city_scores"]

    mami_results.append({
        "config_id": config_id, 
        "response_mami": final_offer, 
        "city_scores_mami": final_city_scores
    })


In [7]:
mami_df = pd.DataFrame(mami_results)
mami_df.head()

,config_id,response_mami,city_scores_mami
0,c_p_98_pop_medium_medium,"[Innsbruck, Bern, Salzburg, Sarajevo, Ljubljan...","{'Aalborg': 0.0, 'Adana': 0.0, 'Adiyaman': 0.0..."
1,c_p_196_pop_medium_hard,"[Innsbruck, Bern, Bergen, Salzburg, Ljubljana,...","{'Adana': 0.0, 'Adiyaman': 0.0, 'Agri': 0.0, '..."
2,c_p_24_pop_low_medium,"[Ljubljana, Sarajevo, Perugia, Plovdiv, Coimbr...","{'Adana': 0.0, 'Adiyaman': 0.0, 'Amsterdam': 6..."
3,c_p_77_pop_low_medium,"[Brno, Cluj-Napoca, Poznan, Lodz, Iasi, Braga,...","{'Aalborg': 0.0, 'Adana': 0.0, 'Adiyaman': 0.0..."
4,c_p_133_pop_medium_medium,"[Bologna, Strasbourg, Maastricht, Gdansk, Dres...","{'Adana': 0.0, 'Adiyaman': 0.0, 'Agri': 0.0, '..."


In [ ]:
gemini_results_df = pd.merge(
    left=sasi_gemini,
    right=mami_df, 
    on=["config_id"],
    how="inner"
)

In [ ]:
gemini_results_df.head()

In [ ]:
configs_df = pd.read_json(f"../../data/conv-trs/kg-generation/new-kg/configs/retrieved_filtered_personas_configs.json")
def get_retrieved_cities(config_id):
    return configs_df[configs_df['config_id'] == config_id]['cities'].values[0]

In [ ]:
gemini_results_df["context_cities"] = gemini_results_df['config_id'].apply(get_retrieved_cities)

In [ ]:
gemini_results_df.head()

In [ ]:
gemini_results_df['response_Gemini2Flash'] = gemini_results_df['response_Gemini2Flash'].apply(ast.literal_eval)

### Computing hit rate 

In [ ]:
low_pop_df = gemini_results_df[gemini_results_df['config_id'].str.contains("pop_low")]

In [ ]:
# low_pop_df=gemini_results_df

In [ ]:
# Compute intersection size for each row
low_pop_df['intersection_sasi'] = low_pop_df.apply(
    lambda row: len(set(row['response_Gemini2Flash']).intersection(row['context_cities'])), axis=1
)

# Compute the average intersection size
average_intersection = low_pop_df['intersection_sasi'].mean()

print("Average Intersection between SASI results and Context Cities for Gemini:", average_intersection)

In [ ]:
# Compute intersection size for each row
low_pop_df['intersection_mami'] = low_pop_df.apply(
    lambda row: len(set(row['response_mami']).intersection(row['context_cities'])), axis=1
)

# Compute the average intersection size
average_intersection = low_pop_df['intersection_mami'].mean()

print("Average Intersection between MAMI results and Context Cities for Gemini:", average_intersection)